## Importação da Bibliotecas

In [3]:
import datetime
from datetime import datetime

# Base de dados
import pandas as pd
import numpy as np

# Documento
from docx import Document
from docx.shared import Inches
from docx.enum.style import WD_STYLE_TYPE
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Pt, RGBColor, Inches

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import plotly.express as px
import plotly.io as pio

In [4]:
def gerar_relatorio_uf(uf):
    def titulo(titulo=''):
        t = doc_subsidio.add_paragraph()
        tt = t.add_run(titulo)
        tt.bold = True

    def lista(texto=''):
        b = doc_subsidio.add_paragraph(style='List Bullet')
        b.add_run(texto)

    def var_perc(x, y):
        return f'{(abs((y - x) / x)) * 100:.1f}%'

    def rep_perc(x, y):
        return f'{(x / y) * 100:.1f}%'
    
    fFeminicidio = pd.read_excel('Dados/Dados - ABSP.xlsx', sheet_name='fFeminicidio')
    fFeminicidioCapital = pd.read_excel('Dados/Dados - ABSP.xlsx', sheet_name='fFeminicidioCapital')
    dLocalidade = pd.read_excel('Dados/Dimensões.xlsx', sheet_name='dLocalidade')

    fFeminicidio = fFeminicidio.merge(dLocalidade, how='left', on='cod_uf')
    fFeminicidio = fFeminicidio.drop(columns='cod_uf')
    fFeminicidio = fFeminicidio.loc[fFeminicidio['tipo_de_crime'] == 'Feminicídio Consumado']
    l = ['taxa_100mil_hab', 'qtd']
    for k, v in enumerate(l):
        fFeminicidio[f'{v}'].fillna(0, inplace=True)
        if k > 0:
            fFeminicidio[f'{v}'] = fFeminicidio[f'{v}'].astype('int64')
        else:
            fFeminicidio[f'{v}'] = fFeminicidio[f'{v}'].replace('...', np.nan)
            fFeminicidio[f'{v}'] = pd.to_numeric(fFeminicidio[f'{v}'])

    fFeminicidioCapital = fFeminicidioCapital.merge(dLocalidade, how='left', on='cod_uf')
    fFeminicidioCapital = fFeminicidioCapital.drop(columns='cod_uf')
    l = ['taxa_100mil_hab', 'qtd']
    for k, v in enumerate(l):
        fFeminicidioCapital[f'{v}'].fillna(0, inplace=True)
        if k > 0:
            fFeminicidioCapital[f'{v}'] = fFeminicidioCapital[f'{v}'].replace('...', np.nan).replace('-', np.nan)
            fFeminicidioCapital[f'{v}'] = fFeminicidioCapital[f'{v}'].fillna(0)
            fFeminicidioCapital[f'{v}'] = fFeminicidioCapital[f'{v}'].astype('int64')
        else:
            fFeminicidioCapital[f'{v}'] = fFeminicidioCapital[f'{v}'].replace('...', np.nan).replace('-', np.nan)
            fFeminicidioCapital[f'{v}'] = fFeminicidioCapital[f'{v}'].fillna(0)
            fFeminicidioCapital[f'{v}'] = pd.to_numeric(fFeminicidioCapital[f'{v}'])

    fFeminicidio['taxa_100mil_hab'] = round(fFeminicidio['taxa_100mil_hab'], 2)
    fFeminicidioCapital['taxa_100mil_hab'] = round(fFeminicidioCapital['taxa_100mil_hab'], 2)

    ufs_extenso = {'AC': 'Acre', 'AL': 'Alagoas', 'AP': 'Amapá', 'AM': 'Amazonas', 'BA': 'Bahia', 'CE': 'Ceará',
                        'DF': 'Distrito Federal', 'ES': 'Espírito Santo', 'GO': 'Goiás', 'MA': 'Maranhão',
                        'MT': 'Mato Grosso', 'MS': 'Mato Grosso do Sul', 'MG': 'Minas Gerais', 'PA': 'Pará',
                        'PB': 'Paraíba',
                        'PR': 'Paraná', 'PE': 'Pernambuco', 'PI': 'Piauí', 'RJ': 'Rio de Janeiro',
                        'RN': 'Rio Grande do Norte', 'RS': 'Rio Grande do Sul', 'RO': 'Rondônia', 'RR': 'Roraima',
                        'SC': 'Santa Catarina', 'SP': 'São Paulo', 'SE': 'Sergipe', 'TO': 'Tocantins'}

    ufs_regioes = {'AC': 'Norte', 'AL': 'Nordeste', 'AP': 'Norte', 'AM': 'Norte', 'BA': 'Nordeste', 'CE': 'Nordeste',
                            'DF': 'Centro-Oeste', 'ES': 'Sudeste', 'GO': 'Centro-Oeste', 'MA': 'Nordeste',
                            'MT': 'Centro-Oeste', 'MS': 'Centro-Oeste', 'MG': 'Sudeste', 'PA': 'Norte',
                            'PB': 'Nordeste',
                            'PR': 'Sul', 'PE': 'Nordeste', 'PI': 'Nordeste', 'RJ': 'Sudeste',
                            'RN': 'Nordeste', 'RS': 'Sul', 'RO': 'Norte', 'RR': 'Norte',
                            'SC': 'Sul', 'SP': 'Sudeste', 'SE': 'Nordeste', 'TO': 'Norte'}

    ufs = [c for c in dLocalidade['uf']]
    capital = [c for c in dLocalidade['capital']]
    ufs_capital = {}
    for k, v in enumerate(capital):
        ufs_capital[f'{ufs[k]}'] = v 

    uf_extenso = ufs_extenso[uf]
    uf_regiao = ufs_regioes[uf]
    uf_capital = ufs_capital[uf]
    fFeminicidio_uf = fFeminicidio.loc[fFeminicidio['uf'] == uf]
    
    fig1 = px.line(fFeminicidio_uf, x='ano', y='qtd', title=f'Evolução dos Feminicídios - {uf_extenso}', markers=True, text='qtd',
                            width=1200, height=450)
    fig1.update_traces(textposition='top center')
    fig1.update_layout(xaxis_title='Ano', yaxis_title='Quantidade')
    pio.write_image(fig1, '1.png')

    fFeminicidio_uf_br = fFeminicidio.loc[((fFeminicidio['uf'] == uf) | (fFeminicidio['uf'] == 'BR')) & (fFeminicidio['ano'] == 2021)].copy()
    fFeminicidio_uf_br.sort_values(by='qtd', inplace=True)

    fig2 = px.bar(fFeminicidio_uf_br, x='uf_extenso', y='qtd', text='qtd', title=f'{uf_extenso} x Brasil - Feminicídios em 2021', width=1200, height=450)
    fig2.update_layout(xaxis_title='Localidade', yaxis_title='Quantidade')
    pio.write_image(fig2, '2.png')

    fig3 = px.bar(fFeminicidio_uf_br, x='uf_extenso', y='taxa_100mil_hab', text='taxa_100mil_hab',
                 title=f'{uf_extenso} x Brasil - Taxa de Feminicídios em 2021', width=1200, height=450)
    fig3.update_layout(xaxis_title='Localidade', yaxis_title='Taxa por 100 mil mulheres')
    pio.write_image(fig3, '3.png')

    fFeminicidio_regiao = fFeminicidio.loc[(fFeminicidio['ano'] == 2021) & (fFeminicidio['uf'] != 'BR')]
    d1 = fFeminicidio_regiao[['regiao', 'qtd']].groupby('regiao').sum().reset_index()
    valor_regiao = int(d1.loc[d1['regiao'] == f'{uf_regiao}']['qtd'])

    d2 = fFeminicidio_uf.loc[fFeminicidio['ano'] == 2021]
    valor_uf = int(d2['qtd'])
    d3 = fFeminicidio_uf.loc[fFeminicidio['ano'] == 2020]
    valor_uf_2020 = int(d3['qtd'])
    valor_uf_taxa = float(d2['taxa_100mil_hab'])

    data_uf_regiao = pd.DataFrame({'Localidade': [f'{uf_extenso}', f'{uf_regiao}'],
                                    'Quantidade': [valor_uf, valor_regiao]})

    fig4 = px.bar(data_uf_regiao, x='Localidade', y='Quantidade', text='Quantidade', width=1200, height=450
                ,title=f'{uf_extenso} x {uf_regiao} - Feminicídios em 2021')
    pio.write_image(fig4, '4.png')

    data_uf_uf = fFeminicidio.loc[(fFeminicidio['ano'] == 2021) & (fFeminicidio['uf'] != 'BR')][['uf', 'qtd', 'taxa_100mil_hab']]

    default_color = 'blue'
    colors_uf = {f'{uf}': 'red'}

    color_discrete_map_uf = {
        c: colors_uf.get(c, default_color)
        for c in data_uf_uf['uf'].unique()
    }

    fig5 = px.bar(data_uf_uf.sort_values(by='qtd'), x='uf', y='qtd', text='qtd', color_discrete_map=color_discrete_map_uf, color='uf',
                title=f'{uf_extenso} x UFs - Feminicídios em 2021', width=1200, height=450)
    fig5.update_layout(xaxis_title='UF', yaxis_title='Quantidade')
    fig5.update_traces(showlegend=False)
    pio.write_image(fig5, '5.png')

    fig6 = px.bar(data_uf_uf.sort_values(by='taxa_100mil_hab'), x='uf', y='taxa_100mil_hab', text='taxa_100mil_hab', 
                                    color_discrete_map=color_discrete_map_uf, color='uf', width=1200, height=450
                                    ,title=f'{uf_extenso} x UFs - Taxa de Feminicídios em 2021')
    fig6.update_layout(xaxis_title='UF', yaxis_title='Taxa por 100 mil mulheres')
    fig6.update_traces(showlegend=False)
    pio.write_image(fig6, '6.png')

    valor_capital = int(fFeminicidioCapital.loc[(fFeminicidioCapital['ano'] == 2021) & (fFeminicidioCapital['capital'] == f'{uf_capital}')]['qtd'])
    data_uf_capital = pd.DataFrame({'Localidade': [f'{uf_capital}', f'{uf}'], 'Quantidade': [valor_capital, valor_uf]})

    fig7 = px.bar(data_uf_capital, x='Localidade', y='Quantidade', text='Quantidade', width=1200, height=450,
                title=f'{uf_capital} x {uf} - Feminicídios em 2021')
    pio.write_image(fig7, '7.png')

    valor_capital_taxa = float(fFeminicidioCapital.loc[(fFeminicidioCapital['ano'] == 2021) & 
                                                (fFeminicidioCapital['capital'] == f'{uf_capital}')]['taxa_100mil_hab'])
    data_uf_capital = pd.DataFrame({'Localidade': [f'{uf_capital}', f'{uf}'], 'Taxa por 100 mil mulheres': [valor_capital_taxa, valor_uf_taxa]})

    fig8 = px.bar(data_uf_capital, x='Localidade', y='Taxa por 100 mil mulheres', text='Taxa por 100 mil mulheres',
                title=f'{uf_capital} x {uf} - Taxa de Feminicídios em 2021', width=1200, height=450)
    pio.write_image(fig8, '8.png')

    data_capital_capital = fFeminicidioCapital.loc[(fFeminicidioCapital['ano'] == 2021) & 
                                               (fFeminicidioCapital['uf'] != 'BR')][['uf', 'capital', 'qtd', 'taxa_100mil_hab']]

    data_capital_capital['capital/uf'] = data_capital_capital['capital'] + '/' + data_capital_capital['uf']
    colors = {f'{uf_capital}/{uf}': 'red'}

    color_discrete_map_capital = {
        c: colors.get(c, default_color)
        for c in data_capital_capital['capital/uf'].unique()
    }

    fig9 = px.bar(data_capital_capital.sort_values(by='qtd'), x='capital/uf', y='qtd', color_discrete_map=color_discrete_map_capital,
                text='qtd', color='capital/uf', width=1200, height=450,
                title=f'{uf_capital} x Capitais - Feminicídios em 2021')
    fig9.update_layout(xaxis_title='Capital', yaxis_title='Quantidade')
    fig9.update_traces(showlegend=False)
    pio.write_image(fig9, '9.png')

    fig10 = px.bar(data_capital_capital.sort_values(by='taxa_100mil_hab'), x='capital/uf', y='taxa_100mil_hab', 
             color_discrete_map=color_discrete_map_capital, text='taxa_100mil_hab', color='capital/uf',
            title=f'{uf_capital} x Capitais - Taxa de Feminicídios em 2021', width=1200, height=450)
    fig10.update_layout(xaxis_title='Capital', yaxis_title='Taxa por 100 mil mulheres')
    fig10.update_traces(showlegend=False)
    pio.write_image(fig10, '10.png')
    
    doc_subsidio = Document()
    style = doc_subsidio.styles['Normal']
    font = style.font

    font.name = 'Calibri'
    font.size = Pt(12)

    doc_subsidio.add_picture('imagens\governo_federal_logo.png', width=Inches(0.8), height=Inches(0.8))
    last_paragraph = doc_subsidio.paragraphs[-1] 
    last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    cabecalho = doc_subsidio.add_paragraph()
    cabecalho.alignment = WD_ALIGN_PARAGRAPH.CENTER
    p = cabecalho.add_run(f'MINISTÉRIO DA MULHER, DA FAMÍLIA E DOS DIREITOS HUMANOS (MMFDH) \nSECRETARIA NACIONAL DE POLÍTICAS PARA MULHERES (SNPM) \n \n{ufs_extenso[uf].upper()}') 
    p.bold = True
    doc_subsidio.add_paragraph()

    c = doc_subsidio.add_paragraph()
    p = c.add_run('Feminicídios - Fórum Brasileiro de Segurança Pública')
    p.bold = True
    p.font.size = Pt(14)
    
    palavra_var = 'um aumento'
    if valor_uf< valor_uf_2020:
        palabra_var = 'uma redução'

    titulo('Evolução')
    doc_subsidio.add_picture(f"1.png", width=Inches(6))
    lista(f'Em relação a 2020, houve {palavra_var} de {var_perc(valor_uf_2020, valor_uf)} no ano de 2021.')

    vbrasil_2021 = fFeminicidio_uf_br.iloc[1]['qtd']

    titulo(f'{uf_extenso} x Brasil')
    doc_subsidio.add_picture(f"2.png", width=Inches(6))
    lista(f'Os feminicídios em {uf} representam {rep_perc(valor_uf, vbrasil_2021)} dos feminicídios do Brasil.')

    valor_br_taxa = fFeminicidio_uf_br.iloc[1]['taxa_100mil_hab']

    doc_subsidio.add_picture(f"3.png", width=Inches(6))
    lista(f'No Brasil, morreram {valor_br_taxa} mulheres por feminicídio em cada 100 mil mulheres.')
    lista(f'Em {uf_extenso}, morreram {valor_uf_taxa} mulheres por feminicídio em cada 100 mil mulheres.')

    titulo(f'{uf_extenso} x {uf_regiao}')
    doc_subsidio.add_picture(f"4.png", width=Inches(6))
    lista(f'Os feminicídios em {uf_extenso} representam {rep_perc(valor_uf, valor_regiao)} dos feminicídios no {uf_regiao}.')

    data_uf_uf_numero = data_uf_uf.sort_values(by='qtd', ascending=False).reset_index(drop=True).reset_index()
    data_uf_uf_numero['index'] = data_uf_uf_numero["index"] + 1

    titulo(f'{uf_extenso} x UFs')
    doc_subsidio.add_picture(f"5.png", width=Inches(6))
    lista(f'Em questão de número de feminicídios, {uf_extenso} se encontra na {int(data_uf_uf_numero.loc[data_uf_uf_numero["uf"] == uf]["index"])}° posição.')

    data_uf_uf_taxa = data_uf_uf.sort_values(by='taxa_100mil_hab', ascending=False).reset_index(drop=True).reset_index()
    data_uf_uf_taxa['index'] = data_uf_uf_taxa["index"] + 1

    doc_subsidio.add_picture(f"6.png", width=Inches(6))
    lista(f'Em questão de taxa de feminicídio (por 100 mil mulheres), {uf_extenso} se encontra na {int(data_uf_uf_taxa.loc[data_uf_uf_taxa["uf"] == uf]["index"])}° posição.')

    valor_capital_n_2020 = fFeminicidioCapital.loc[fFeminicidioCapital['uf'] == uf].iloc[0]['qtd']
    valor_capital_n_2021 = fFeminicidioCapital.loc[fFeminicidioCapital['uf'] == uf].iloc[1]['qtd']

    valor_capital_t_2020 = fFeminicidioCapital.loc[fFeminicidioCapital['uf'] == uf].iloc[0]['taxa_100mil_hab']
    valor_capital_t_2021 = fFeminicidioCapital.loc[fFeminicidioCapital['uf'] == uf].iloc[1]['taxa_100mil_hab']

    palavra_var = 'um aumento'
    if valor_capital_n_2020 > valor_capital_n_2021:
        palabra_var = 'uma redução'

    titulo(f'{uf_capital} x {uf_extenso}')
    doc_subsidio.add_picture(f"7.png", width=Inches(6))
    lista(f'Em {uf_extenso}, cerca de {rep_perc(valor_capital, valor_uf)} dos feminicídios aconteceram na sua capital.')
    lista(f'Em relação a 2020 ({valor_capital_n_2020} feminicídios na capital), houve um {palavra_var} de {var_perc(valor_capital_n_2020, valor_capital_n_2021)} do número de feminicídios na sua capital em 2021.')

    doc_subsidio.add_picture(f"8.png", width=Inches(6))
    lista(f'Em {uf_extenso}, morreram {valor_uf_taxa} mulheres por feminicídio em cada 100 mil mulheres.')
    lista(f'Em sua capital, {uf_capital}, morreram {valor_capital_t_2021} mulheres por feminicídio em cada 100 mil mulheres.')

    data_capital_capital_n = data_capital_capital.sort_values(by='qtd', ascending=False).reset_index(drop=True).reset_index()
    data_capital_capital_n['index'] = data_uf_uf_numero["index"]

    titulo(f'{uf_capital} x Capitais')
    doc_subsidio.add_picture(f"9.png", width=Inches(6))
    lista(f'Em comparação com as outras capitais em número de feminicídio, {uf_capital} se encontra na {int(data_capital_capital_n.loc[data_capital_capital_n["uf"] == uf]["index"])}° posição.')

    data_capital_capital_t = data_capital_capital.sort_values(by='taxa_100mil_hab', ascending=False).reset_index(drop=True).reset_index()
    data_capital_capital_t['index'] = data_uf_uf_numero["index"]

    doc_subsidio.add_picture(f"10.png", width=Inches(6))
    lista(f'Em comparação com as outras capitais em taxa de feminicídio, {uf_capital} se encontra na {int(data_capital_capital_t.loc[data_capital_capital_t["uf"] == uf]["index"])}° posição.')

    d = datetime.now()
    doc_subsidio.add_paragraph()
    doc_subsidio.add_paragraph()
    doc_subsidio.add_paragraph('Fonte: Anuário de Segurança Pública (Fórum Brasileiro de Segurança Pública)')
    doc_subsidio.add_paragraph('Data de coleta dos dados: 16/11/2022')
    doc_subsidio.add_paragraph(f'Data de geração do relatório: {d.day}/{d.month}/{d.year}')

    doc_subsidio.save(f'Relatórios gerados/{uf.upper()}_relatório.docx')

In [5]:
estados = {'AC': 'Acre', 'AL': 'Alagoas', 'AP': 'Amapá', 'AM': 'Amazonas', 'BA': 'Bahia', 'CE': 'Ceará',
                        'DF': 'Distrito Federal', 'ES': 'Espírito Santo', 'GO': 'Goiás', 'MA': 'Maranhão',
                        'MT': 'Mato Grosso', 'MS': 'Mato Grosso do Sul', 'MG': 'Minas Gerais', 'PA': 'Pará',
                        'PB': 'Paraíba',
                        'PR': 'Paraná', 'PE': 'Pernambuco', 'PI': 'Piauí', 'RJ': 'Rio de Janeiro',
                        'RN': 'Rio Grande do Norte', 'RS': 'Rio Grande do Sul', 'RO': 'Rondônia', 'RR': 'Roraima',
                        'SC': 'Santa Catarina', 'SP': 'São Paulo', 'SE': 'Sergipe', 'TO': 'Tocantins'}

In [8]:
for k in estados.keys():
    gerar_relatorio_uf(k)

C:\Users\renan.santos\AppData\Local\Temp\ipykernel_4104\2534566128.py:12: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\renan.santos\AppData\Local\Temp\ipykernel_4104\2534566128.py:12: RuntimeWarning:

divide by zero encountered in longlong_scalars

